# Concevez une application au service de la santé publique.

Idée d'application : lorsque l'utilisateur scanne un produit, l'application lui propose une alternative sans allergène, la plus saine possible (peu d'additifs, sans huile de palme avec un bon Nutri-Score : faible en énergie, en glucide, en graisse, en sel, riche en fibres et en protéines) vendue en France.

## Nettoyage du jeu de données.

### 1. Filtrage du dataset : 

J'importe les librairies nécessaires.

In [1]:
import pandas as pd 
import numpy as np
import os
from sklearn.impute import KNNImputer

J'utilise la fonction "read" pour lire le fichier csv fourni. J'utilise l'argument "sep='\t'", car les éléments de mon fichier sont séparés par des tabulations et non par des virgules.

In [2]:
data = pd.read_csv("fr.openfoodfacts.org.products.csv",sep='\t')
data.head()

/home/elena/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,35,36,37,38,39,48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,3087,http://world-fr.openfoodfacts.org/produit/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4530,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.0,NaN,NaN
2,4559,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Peanuts,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
3,16087,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055731,2017-03-09T10:35:31Z,1489055731,2017-03-09T10:35:31Z,Organic Salted Nut Mix,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12.0,NaN,NaN
4,16094,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055653,2017-03-09T10:34:13Z,1489055653,2017-03-09T10:34:13Z,Organic Polenta,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Je filtre mon DataFrame en gardant les variables qui m'intéressent. 

In [3]:
data = data.loc[:, ["code", "product_name", "countries_fr", "main_category_fr", "allergens", "nutrition_grade_fr", "additives_n", "ingredients_from_palm_oil_n", "energy_100g", "proteins_100g", "carbohydrates_100g", "fat_100g","sodium_100g", "fiber_100g"]]
data.head()

,code,product_name,countries_fr,main_category_fr,allergens,nutrition_grade_fr,additives_n,ingredients_from_palm_oil_n,energy_100g,proteins_100g,carbohydrates_100g,fat_100g,sodium_100g,fiber_100g
0,3087,Farine de blé noir,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4530,Banana Chips Sweetened (Whole),États-Unis,NaN,NaN,d,0.0,0.0,2243.0,3.57,64.29,28.57,0.000,3.6
2,4559,Peanuts,États-Unis,NaN,NaN,b,0.0,0.0,1941.0,17.86,60.71,17.86,0.250,7.1
3,16087,Organic Salted Nut Mix,États-Unis,NaN,NaN,d,0.0,0.0,2540.0,17.86,17.86,57.14,0.482,7.1
4,16094,Organic Polenta,États-Unis,NaN,NaN,NaN,0.0,0.0,1552.0,8.57,77.14,1.43,NaN,5.7


Je séléctionne seulement les produits vendus en France.

In [4]:
data = data.loc[(data)["countries_fr"] == "France"]
data

,code,product_name,countries_fr,main_category_fr,allergens,nutrition_grade_fr,additives_n,ingredients_from_palm_oil_n,energy_100g,proteins_100g,carbohydrates_100g,fat_100g,sodium_100g,fiber_100g
0,3087,Farine de blé noir,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,24600,Filet de bœuf,France,Filet-de-boeuf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,27205,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136,39259,Twix x2,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,5200016,lentilles vertes,France,Aliments et boissons à base de végétaux,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320761,9906410000009,Roussette du Bugey (2011),France,Boissons,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320763,99111250,Thé vert Earl grey,France,Thés verts,NaN,c,0.0,0.0,21.0,0.5,0.5,0.2,0.01,0.2
320764,9918,"Cheese cake thé vert, yuzu",France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320765,9935010000003,Rillette d'oie,France,Produits à tartiner salés,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


### 2. Erreurs de type :

Je vais commencer mon nettoyage. Je vérifie que mes données ont été implémentées avec le bon type.

In [5]:
data.dtypes

code                            object
product_name                    object
countries_fr                    object
main_category_fr                object
allergens                       object
nutrition_grade_fr              object
additives_n                    float64
ingredients_from_palm_oil_n    float64
energy_100g                    float64
proteins_100g                  float64
carbohydrates_100g             float64
fat_100g                       float64
sodium_100g                    float64
fiber_100g                     float64
dtype: object

### 3. Doublons :

Je vérifie si y a des données doublées dans mon DataFrame grâce à mes indicateurs.

In [6]:
data.loc[data[["product_name", "countries_fr", "main_category_fr", "allergens", "nutrition_grade_fr", "additives_n", "ingredients_from_palm_oil_n", "energy_100g", "proteins_100g", "carbohydrates_100g", "fat_100g","sodium_100g", "fiber_100g"]].duplicated(keep=False),:]

,code,product_name,countries_fr,main_category_fr,allergens,nutrition_grade_fr,additives_n,ingredients_from_palm_oil_n,energy_100g,proteins_100g,carbohydrates_100g,fat_100g,sodium_100g,fiber_100g
0,3087,Farine de blé noir,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,27205,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187,9336247,NaN,France,Bonbons,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,50157846,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
271,228000080,Terrine de campagne,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320745,9800801107,NaN,France,Pâtes à tartiner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320746,9800895007,NaN,France,Pâtes à tartiner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320747,9800895250,NaN,France,Pâtes à tartiner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320755,988639,NaN,France,Pâtes à tartiner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Je décide de supprimer tous les doublons présents dans mon DataFrame.

In [7]:
data.drop_duplicates(subset=["product_name", "countries_fr", "main_category_fr", "allergens", "nutrition_grade_fr", "additives_n", "ingredients_from_palm_oil_n", "energy_100g", "proteins_100g", "carbohydrates_100g", "fat_100g","sodium_100g", "fiber_100g"], inplace=True, ignore_index=True)

### 4. Détéction d'outliers : 

Je regarde si il y a des outliers.

In [8]:
data.describe()

,additives_n,ingredients_from_palm_oil_n,energy_100g,proteins_100g,carbohydrates_100g,fat_100g,sodium_100g,fiber_100g
count,49280.000000,49280.000000,6.084900e+04,60599.000000,43658.000000,44067.000000,58962.000000,43313.000000
mean,1.883076,0.069176,1.168596e+03,7.804283,27.388828,13.238258,0.458482,2.526598
std,2.577834,0.257645,1.322325e+04,7.927826,27.295711,16.811237,1.696839,4.642176
min,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,4.270000e+02,1.800000,4.000000,1.300000,0.031496,0.000000
50%,1.000000,0.000000,1.029000e+03,6.000000,14.100000,6.800000,0.224409,1.300000
75%,3.000000,0.000000,1.639000e+03,11.000000,52.800000,21.000000,0.492126,3.200000
max,31.000000,2.000000,3.251373e+06,100.000000,190.000000,380.000000,83.000000,178.000000


Je vais traiter les outliers évidents. Je commence par regarder les variables "additives_n" et "energy_100g".

In [9]:
data.loc[((data["additives_n"] == 31) | (data["energy_100g"] == 3.251373e+06)), :]

,code,product_name,countries_fr,main_category_fr,allergens,nutrition_grade_fr,additives_n,ingredients_from_palm_oil_n,energy_100g,proteins_100g,carbohydrates_100g,fat_100g,sodium_100g,fiber_100g
6487,2600280009649,Confiserie,France,Snacks sucrés,"BLE, BLE, LACTOSE, LAIT, LAIT",d,31.0,1.0,1702.0,1.9,83.2,6.9,0.062992,NaN
6488,2600280012205,Confiserie,France,Bonbons,"BLE, BLE, LACTOSE, LAIT, LAIT",d,31.0,1.0,1702.0,1.8,83.2,6.9,0.062992,NaN
30513,3257983143096,Pois cassés,France,Pois-casses,NaN,b,0.0,0.0,3251373.0,22.7,48.2,1.2,0.003937,15.4


additives_n : Les produits concernés sont des confiseries, il n'est donc pas improbable qu'ils contiennent 31 additifs. Ces outliers sont donc des valeurs atypiques, que je décide de garder.

energy_100g : Le produit concerné est une boîte de pois cassés, il est très surprenant qu'il contienne autant de calories, je décide de le supprimer. Cet outlier est une valeur aberrante.

In [10]:
data = data.drop(30513)

Toutes les valeurs que ne sont pas comprises entre 0 et 100 pour les valeurs nutritionnelles : "sodium_100g", "proteins_100g", "carbohydrates_100g", "fat_100g","fiber_100g" sont des outliers. En effet, ces indicateurs ne peuvent pas être supérieurs à 100 puisque l'étude est faite sur un échantillon de 100g. De même ils ne peuvent pas être inférieurs à 0. Ces outliers sont donc des valeurs aberrantes que je décide de supprimer.
La variable "energy_100g" n'est pas concernée par ce raisonnement car elle n'est pas exprimée en g mais en kJ.

Je supprime les valeurs aberrantes en récupérant leurs index. Je parcours mon DataFrame pour récupérer toutes les valeurs aberrantes des valeurs nutritionnelles si jamais la base de données est légèrement modifiée (ajout ou suppression d'entrées par exemple).

In [11]:
for i in data.index:
    if (data["proteins_100g"][i] > 100) or (data["proteins_100g"][i] < 0) or (data["carbohydrates_100g"][i] > 100) or (data["carbohydrates_100g"][i] < 0) or (data["fat_100g"][i] > 100) or (data["fat_100g"][i] < 0) or (data["sodium_100g"][i] > 100) or (data["sodium_100g"][i] < 0) or (data["fiber_100g"][i] > 100) or (data["fiber_100g"][i] < 0):
        data = data.drop(i)

Je vérifie si tout est bon.

In [12]:
data.describe()

,additives_n,ingredients_from_palm_oil_n,energy_100g,proteins_100g,carbohydrates_100g,fat_100g,sodium_100g,fiber_100g
count,49273.000000,49273.000000,60838.000000,60589.000000,43647.000000,44057.000000,58953.000000,43306.000000
mean,1.883141,0.069166,1114.772736,7.804630,27.368478,13.228366,0.458474,2.522471
std,2.577855,0.257628,1111.874432,7.927913,27.261322,16.714515,1.696911,4.564846
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,427.000000,1.800000,4.000000,1.300000,0.031496,0.000000
50%,1.000000,0.000000,1029.000000,6.000000,14.000000,6.800000,0.224409,1.300000
75%,3.000000,0.000000,1638.000000,11.000000,52.800000,21.000000,0.492126,3.200000
max,31.000000,2.000000,182764.000000,100.000000,100.000000,100.000000,83.000000,100.000000


Le max de la variable "energy_100g" me parait encore élevé, je regarde plus en détails.

In [13]:
data.loc[(data["energy_100g"] == 182764.000000)]

,code,product_name,countries_fr,main_category_fr,allergens,nutrition_grade_fr,additives_n,ingredients_from_palm_oil_n,energy_100g,proteins_100g,carbohydrates_100g,fat_100g,sodium_100g,fiber_100g
62207,3661405001053,"Légume Noisettes Carottes, 500 Grammes, Marque...",France,NaN,NaN,d,NaN,NaN,182764.0,4.0,21.0,9.0,0.468504,3.5


Je constate que ce produit correspond à des légumes, je décide de supprimer cet outlier qui est une valeur aberrante.

In [14]:
data = data.drop(62207)

Je vérifie si tout est bon.

In [15]:
data.describe()

,additives_n,ingredients_from_palm_oil_n,energy_100g,proteins_100g,carbohydrates_100g,fat_100g,sodium_100g,fiber_100g
count,49273.000000,49273.000000,60837.000000,60588.000000,43646.000000,44056.000000,58952.000000,43305.000000
mean,1.883141,0.069166,1111.786901,7.804693,27.368624,13.228462,0.458474,2.522448
std,2.577855,0.257628,833.002204,7.927963,27.261618,16.714693,1.696925,4.564896
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,427.000000,1.800000,4.000000,1.300000,0.031496,0.000000
50%,1.000000,0.000000,1029.000000,6.000000,14.000000,6.800000,0.224409,1.300000
75%,3.000000,0.000000,1638.000000,11.000000,52.800000,21.000000,0.492126,3.200000
max,31.000000,2.000000,69292.000000,100.000000,100.000000,100.000000,83.000000,100.000000


Le max de la variable 'energy_100g' est encore trop élevé.

In [16]:
data.loc[data["energy_100g"] == 69292]

,code,product_name,countries_fr,main_category_fr,allergens,nutrition_grade_fr,additives_n,ingredients_from_palm_oil_n,energy_100g,proteins_100g,carbohydrates_100g,fat_100g,sodium_100g,fiber_100g
36303,3270160689774,NaN,France,NaN,NaN,c,NaN,NaN,69292.0,0.8,14.8,0.5,0.003937,3.4


Je ne sais pas à quel produit cela correspond, mais il a un nutri score c (cela n'est pas cohérent avec une énergie si élevée). Je décide de supprimer cet outlier qui est une valeurs aberrante.

In [17]:
data = data.drop(36303)

Je vérifie que les nutri score soient bien compris entre 'a' et 'e'

In [18]:
data.groupby("nutrition_grade_fr").count()

,code,product_name,countries_fr,main_category_fr,allergens,additives_n,ingredients_from_palm_oil_n,energy_100g,proteins_100g,carbohydrates_100g,fat_100g,sodium_100g,fiber_100g
nutrition_grade_fr,,,,,,,,,,,,,
a,9713,9669,9713,7316,2464,6827,6827,9698,9698,7310,7294,9698,8806
b,8752,8699,8752,5926,2784,5477,5477,8697,8697,5891,5890,8697,6913
c,12519,12462,12519,9094,4027,8300,8300,12519,12519,8925,8926,12519,9244
d,15995,15925,15995,11458,5435,10437,10437,15995,15995,11202,11254,15995,10443
e,10937,10869,10937,7704,3706,6979,6979,10937,10937,7512,7527,10937,7036


### 5. Valeurs manquantes :

Je vérifie si mon jeu de données contient des valeurs manquantes.

In [19]:
data.isnull().sum()

code                               0
product_name                     441
countries_fr                       0
main_category_fr               28252
allergens                      63643
nutrition_grade_fr             27259
additives_n                    35902
ingredients_from_palm_oil_n    35902
energy_100g                    24339
proteins_100g                  24588
carbohydrates_100g             41530
fat_100g                       41120
sodium_100g                    26224
fiber_100g                     41871
dtype: int64

Je décide de remplacer les valeurs manquantes des variables "additives_n" et "ingredients_from_palm_oil_n" avec la méthode des K-nn. 

Je génère un DataFrame x avec les éléments qui m'intéressent.

In [21]:
x = data.loc[:, ("additives_n", "ingredients_from_palm_oil_n", "code")]

J'applique le KNN Imputer avec 2 voisins, puis je convertis le resultat en un DataFrame.

In [22]:
imputer = KNNImputer(n_neighbors = 2)
x = imputer.fit_transform(x)
x = pd.DataFrame(x)

Je renomme mes colonnes.

In [23]:
x = x.rename(columns = {0:"additives_n"})
x = x.rename(columns = {1:"ingredients_from_palm_oil_n"})
x = x.rename(columns = {2:"code"})

Je supprime certaines colonnes de mon DataFrame data pour pouvoir faire une fusion avec la fonction merge.

In [24]:
data = data.drop(columns = ["additives_n", "ingredients_from_palm_oil_n"])
data = pd.merge(data, x, on = "code")

Je regarde où j'en suis avec mes valeurs manquantes.

In [25]:
data.isnull().sum()

code                               0
product_name                     353
countries_fr                       0
main_category_fr               22742
allergens                      53094
nutrition_grade_fr             22167
energy_100g                    19710
proteins_100g                  19940
carbohydrates_100g             34060
fat_100g                       33750
sodium_100g                    21330
fiber_100g                     34565
additives_n                        0
ingredients_from_palm_oil_n        0
dtype: int64

J'aimerais remplacer les autres valeurs manquantes des variables "energy_100g", "proteins_100g", "carbohydrates_100g", "fiber_100g", "sodium_100g" et "fat_100g" par la médiane des autres valeurs. 

In [25]:
data_median = data.fillna({"energy_100g" : (data["energy_100g"].median(skipna=True)), "proteins_100g": (data['proteins_100g'].median(skipna=True)), "carbohydrates_100g" : (data['carbohydrates_100g'].median(skipna=True)), "fat_100g" : (data["fat_100g"].median(skipna=True)), "sodium_100g" : (data["sodium_100g"].median(skipna=True)), "fiber_100g" : (data["fiber_100g"].median(skipna=True))})
data.head()

,code,product_name,countries_fr,main_category_fr,allergens,nutrition_grade_fr,energy_100g,proteins_100g,carbohydrates_100g,fat_100g,sodium_100g,fiber_100g,additives_n,ingredients_from_palm_oil_n
0,3087,Farine de blé noir,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0
1,24600,Filet de bœuf,France,Filet-de-boeuf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.5
2,27205,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.5
3,39259,Twix x2,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.0
4,5200016,lentilles vertes,France,Aliments et boissons à base de végétaux,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


Cette méthode fonctionne, mais je ne trouve pas très pertinent de remplacer les valeurs nutritionnelles manquantes par les médianes associées dans le cadre de ma problématique. Je décide finalement de supprimer toutes les lignes dont il manque au moins une valeur nutritionnelle.

In [26]:
data = data.dropna(subset = ["energy_100g", "proteins_100g", "carbohydrates_100g", "fat_100g","sodium_100g", "fiber_100g"])

In [27]:
data.isnull().sum()

code                               0
product_name                     134
countries_fr                       0
main_category_fr                1175
allergens                      13597
nutrition_grade_fr               358
energy_100g                        0
proteins_100g                      0
carbohydrates_100g                 0
fat_100g                           0
sodium_100g                        0
fiber_100g                         0
additives_n                        0
ingredients_from_palm_oil_n        0
dtype: int64

Je considère que les lignes de la variables "allergens" qui sont égale à NaN signifie qu'il n'y a pas d'allergène dans le produit. 

J'exporte mon DataFrame dans un fichier csv pour pouvoir l'ouvrir dans mon notebook d'exploration.

In [28]:
os.makedirs('data', exist_ok=True)  
data.to_csv('data.csv')  